**DPPy** stands for "DPPs in Python". It is a Python Toolbox for sampling DPPs.

In this notebook, we showcase the DPP samplers featured in DPPy, and highlight some of the tools behind the scene.
  
  [GitHub](https://github.com/guilgautier/DPPy)  
  
  [ReadTheDocs](https://dppy.readthedocs.io) [![Documentation Status](https://readthedocs.org/projects/dppy/badge/?version=latest)](https://dppy.readthedocs.io/en/latest/?badge=latest)
  
  [Travis](https://travis-ci.com/guilgautier/DPPy) [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#I.-Installing-the-package" data-toc-modified-id="I.-Installing-the-package-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>I. Installing the package</a></div><div class="lev1 toc-item"><a href="#II.-Sample-DPPs-with-DPPy" data-toc-modified-id="II.-Sample-DPPs-with-DPPy-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>II. Sample DPPs with DPPy</a></div><div class="lev2 toc-item"><a href="#1.-$\beta$-ensembles" data-toc-modified-id="1.-$\beta$-ensembles-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>1. <span class="MathJax_Preview" style="color: inherit;"><span class="MJXp-math" id="MJXp-Span-1"><span class="MJXp-mi MJXp-italic" id="MJXp-Span-2">β</span></span></span><script type="math/tex" id="MathJax-Element-1">\beta</script>-ensembles</a></div><div class="lev2 toc-item"><a href="#2.-Finite-DPPs" data-toc-modified-id="2.-Finite-DPPs-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>2. Finite DPPs</a></div><div class="lev2 toc-item"><a href="#3.-$k$-DPPs" data-toc-modified-id="3.-$k$-DPPs-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>3. <span class="MathJax_Preview">k</span><script type="math/tex">k</script>-DPPs</a></div><div class="lev2 toc-item"><a href="#4.-Exotic-DPPs" data-toc-modified-id="4.-Exotic-DPPs-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>4. Exotic DPPs</a></div><div class="lev1 toc-item"><a href="#III.-Tools-behind-the-scene" data-toc-modified-id="III.-Tools-behind-the-scene-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>III. Tools behind the scene</a></div>

In [ ]:
%pylab inline

**Note:** If you are using Google Colab' please run the following cell.

In [ ]:
#!rm -r DPPy
#!git clone https://github.com/guilgautier/DPPy.git
#!pip install scipy --upgrade
#!pip install DPPy/.

# I. Installing the package
---

Here are detailed [instructions](https://github.com/guilgautier/DPPy#installation). We use [Travis](https://travis-ci.com/guilgautier/DPPy) to make sure that everything builds correctly
    [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)

💣 If you wish to interact with the source code while running this notebook, please uncomment the following cell.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

# II. Sample DPPs with DPPy
----

The two basic methods of each DPPy object are
**`.sample()`** and 
**`.plot()`**.

## 1. $\beta$-ensembles

$\beta$-ensembles are an important particular case of point processes that appear in random matrix theory, see the [documentation](https://dppy.readthedocs.io/en/latest/continuous_dpps/index.html).

When $\beta=2$, they are projection DPPs.

### a. The circular ensemble

Loosely speaking, the circular ensemble is a repulsive version of points uniformly drawn on the unit circle.

It also characterizes the eigenvalue distribution of random unitary matrices.

In [ ]:
from dppy.beta_ensembles import CircularEnsemble

circular = CircularEnsemble(beta=2)  # beta must be a nonnegative integer, default is beta=2

When $\beta$ increases, the samples tend to "cristallize". You can try several values of $\beta$ in the following cell.

In [ ]:
#@title ##### Use a slider! (on G-Colab only!!)

_beta = 10 #@param {type:'slider', min:0, max:100, step:1}
_size = 30  #@param {type:'slider', min:0, max:100, step:1}

circular.beta = _beta
circular.sample_banded_model(size_N=_size)
circular.plot()

Here is a loop that also demonstrates the cristallization.

In [ ]:
for b in (0, 1, 5, 10):
    circular.beta = b
    circular.sample_banded_model(size_N=30)
    circular.plot()

The circular ensemble can be sampled by diagonalizing dense random matrices when $\beta \in \{0, 1, 2, 4\}$.

In [ ]:
circular.beta = 2
# 1. Plot the eigenvalues, they lie on the unit circle
circular.sample_full_model(size_N=30, haar_mode='Hermite')  # Sample # haar_mode = 'Hermite'/'QR'
circular.plot() # Plot of the eigenvalues

# 2. Histogram of the angle of more points, should look uniform on [0,2pi]
circular.sample_full_model(size_N=1000, haar_mode='Hermite')  # Sample
circular.hist()

They can also be sampled by diagonalizing banded (quindiagonal) matrices  when $\beta \in \mathbb{N}^*$.

In [ ]:
circular.beta = 7
circular.sample_banded_model(size_N=30)
circular.plot()

circular.sample_banded_model(size_N=1000)
circular.hist()

When $\beta=0$, the circular ensemble boils down to i.i.d. uniform points on the unit circle.

In [ ]:
circular.beta = 0
circular.sample_banded_model(size_N=30)
circular.plot()

### b. The Hermite ensemble

Loosely speaking, the Hermite ensemble is a repulsive version of points drawn i.i.d. from a unit Gaussian.

It also characterizes the eigenvalue distribution of symmetric Gaussian matrices.

In [ ]:
from dppy.beta_ensembles import HermiteEnsemble

hermite = HermiteEnsemble(beta=4)  # beta can be >=0, default beta=2

Again, the Hermite ensemble can be sampled by diagonalizing a dense random matrix when $\beta \in \{0, 1, 2, 4\}$.

In [ ]:
hermite.sample_full_model(size_N=500)
# hermite.plot(normalization=True)
hermite.hist(normalization=True)

And again, there is a tridiagonal matrix model that has the same law for its eigenvalues. It is much faster to sample.

In [ ]:
hermite.sample_banded_model(size_N=500)
hermite.hist(normalization=True)

The tridiagonal matrix model can have $\beta \in \mathbb{R}_+$.

In [ ]:
# beta can be >= 0, default beta=2
hermite.beta= 5.43  # Or hermite = HermiteEnsemble(beta=5.43)
# Reference measure is N(mu, sigma^2)
hermite.sample_banded_model(loc=0.0, scale=1.0, size_N=500)
# hermite.plot(normalization=True)
hermite.hist(normalization=True)

When $\beta=0$, the Hermite ensemble is a set of i.i.d. Gaussians.

In [ ]:
# beta can be >= 0, default beta=2
hermite.beta= 0  # Or hermite = HermiteEnsemble(beta=5.43)
# Reference measure is N(mu, sigma^2)
hermite.sample_banded_model(size_N=1000)
# hermite.plot(normalization=True)
hermite.hist(normalization=True)  # True: N(0,2) as in full matrix model

### c. The Laguerre ensemble.

Loosely speaking, the Laguerre ensemble is a repulsive version of points drawn i.i.d. from a Gamma distribution.

It also characterizes the eigenvalue distribution of the covariance matrix built from i.i.d. Gaussian vectors.

In [ ]:
from dppy.beta_ensembles import LaguerreEnsemble

laguerre = LaguerreEnsemble(beta=1)  # beta can be >= 0, default beta=2

There is again a dense matrix model for $\beta \in \{0, 1, 2, 4\}$.

In [ ]:
laguerre.sample_full_model(size_N=500, size_M=800)  # M >= N
# laguerre.plot(normalization=True)
laguerre.hist(normalization=True)

There is again a tridiagonal model that is faster to run.

In [ ]:
laguerre.sample_banded_model(size_N=500, size_M=800)
laguerre.hist(normalization=True)

And, again, the tridiagonal matrix model can feature more general values of the parameter $\beta \in \mathbb{R}_+$.

In [ ]:
laguerre.beta = 2.98  # Or laguerre = LaguerreEnsemble(beta=2.98)
# Reference measure is Gamma(k, theta)
laguerre.sample_banded_model(shape=600, scale=2.0, size_N=400)
# laguerre.plot(normalization=True)
laguerre.hist(normalization=True)

Finally, when $\beta=0$, the Laguerre ensemble boils down to i.i.d. $\Gamma(k,\theta)$ variables. 

In [ ]:
laguerre.beta = 0 
# Reference measure is Gamma(k, theta)
laguerre.sample_banded_model(shape=6, scale=4.0, size_N=1000)
# laguerre.plot(normalization=True)
laguerre.hist(normalization=True)  # True: Gamma(shape, 2) as in full matrix model

### d. The Jacobi ensemble

Loosely speaking, the Jacobi ensemble is a repulsive version of points drawn i.i.d. from a beta law.

In [ ]:
from dppy.beta_ensembles import JacobiEnsemble

jacobi = JacobiEnsemble(beta=2)  # beta can be >= 0, default beta=2

Again, there is a dense matrix model for $\beta \in \{0, 1, 2, 4\}$.

In [ ]:
jacobi.sample_full_model(size_N=400, size_M1=500, size_M2=600)  # M_1, M_2 >= N
# jacobi.plot(normalization=True)
jacobi.hist(normalization=True)

Again, there is a faster tridiagonal model.

In [ ]:
jacobi.sample_banded_model(size_N=400, size_M1=500, size_M2=600)
jacobi.hist(normalization=True)

Again, the tridiagonal model can feature more general values of the parameter $\beta \in \mathbb{R}_+$.

In [ ]:
# beta can be >= 0, default beta=2
jacobi.beta = 3.14  # Or jacobi = JacobiEnsemble(beta=3.14) 
# Reference measure is Beta(a,b)
jacobi.sample_banded_model(a=500, b=300, size_N=400)
# jacobi.plot(normalization=True)
jacobi.hist(normalization=True)

And again, $\beta=0$ boils down to drawing i.i.d. $\operatorname{Beta}(a,b)$ random variables.

In [ ]:
jacobi.beta = 0 
# Reference measure is Beta(a, b)
jacobi.sample_banded_model(a=6, b=4, size_N=1000)
# jacobi.plot(normalization=True)
jacobi.hist(normalization=True)  # True/False: Beta(a, b)

### e. The Ginibre ensemble

This one is defined for $\beta = 2$ only, and is usually defined by its dense matrix model. It corresponds to the eigenvalues of a matrix filled with i.i.d. complex unit Gaussians.

In [ ]:
from dppy.beta_ensembles import GinibreEnsemble

ginibre = GinibreEnsemble()  # beta must be 2 (default)

In [ ]:
ginibre.sample_full_model(size_N=40)
ginibre.plot(normalization=True)

## 2. Finite DPPs

We refer to the [documentation](https://dppy.readthedocs.io/en/latest/finite_dpps/index.html) for more details and definitions.

In [ ]:
from numpy.random import rand, randn
from scipy.linalg import qr

from dppy.finite_dpps import FiniteDPP

### a. Instantiating a `FiniteDPP` object

You can define a finite DPP through its correlation kernel $\mathbf{K}
\in\mathbb{R}^{N\times N}$. In that case, we write $\mathcal{X}\sim DPP(\mathbf{K})$ to mean
$$\qquad \mathbb{P}[S \subseteq \mathcal{X}] = \det \mathbf{K}_S.$$

To illustrate the various ways of defining a `FiniteDPP`, we create a bunch of eigenvalues and eigenvectors for $\mathbf{K}$, which we take here to be symmetric.

In [ ]:
r, N = 10, 10

# Random orthogonal vectors
eig_vecs, _ = qr(randn(N, r), mode='economic')
# Random eigenvalues
eig_vals = rand(r)  # 0< <1
#eig_vals = np.random.randint(2, size=r) # 0 or 1 i.e. projection

You can first define a `FiniteDPP`through its kernel itself, provided
$$\qquad 0 \preceq \mathbf{K} \preceq I$$
to guarantee the existence of the DPP.

In [ ]:
DPP = FiniteDPP('correlation',
                **{'K': eig_vecs*eig_vals.dot(eig_vecs.T)})

Equivalently, you can directly input the eigendecomposition,

$\qquad \mathbf{K} = \sum_{n=1}^{N} \lambda_n u_n u_n^{\top}, \quad 0 \leq \lambda_n \leq 1$.

In [ ]:
DPP = FiniteDPP('correlation',
                **{'K_eig_dec': (eig_vals, eig_vecs)})

In machine learning, it is common to forget about $\mathbf{K}$ and existence conditions by working with the "likelihood" kernel $\mathbf{L}\in\mathbb{R}^{N\times N}$, and define $\mathcal{X}\sim\operatorname{DPP}(\mathbf{L})$ to mean

$$\qquad \mathbb{P}[\mathcal{X} = S] \propto \det \mathbf{L}_S.$$

In `DPPy`, you can define a DPP through its likelihood kernel, provided $\mathbf{L}\succeq 0$.

In [ ]:
DPP = FiniteDPP('likelihood',
                **{'L': eig_vecs*(eig_vals/(1-eig_vals)).dot(eig_vecs.T)})

Again, you can also input the eigendecomposition directly,

$$\mathbf{L} = \sum_{n=1}^{N} \delta_n u_n u_n^{\top}, \quad \delta_n \geq 0.$$

In [ ]:
DPP = FiniteDPP('likelihood',
                **{'L_eig_dec': (eig_vals/(1-eig_vals), eig_vecs)})

Finally, we also include an option to input feature vectors $\Phi \in \mathbb{R}^{r\times N}$, which define $\mathbf{L}$ as 

$$\qquad \mathbf{L}=\Phi^{\top} \Phi.$$

In [ ]:
DPP = FiniteDPP('likelihood',
                **{'L_gram_factor': randn(r, N)})

Users can go back and forth between the $\mathbf{K}$ and $\mathbf{L}$ kernels, since $\mathbf{L}=\mathbf{K}(I-\mathbf{K})^{-1}$.

In [ ]:
DPP = FiniteDPP('correlation', **{'K_eig_dec': (eig_vals, eig_vecs)})
DPP.compute_L()
#print(DPP.L)

And vice versa.

In [ ]:
eig_vals = 4*rand(r)  # >=0
DPP = FiniteDPP('likelihood', **{'L_eig_dec': (eig_vals, eig_vecs)})
DPP.compute_K()
#print(DPP.K)

You can also visualize the underlying kernel.

In [ ]:
eig_vals = rand(r)  # 0< <1
DPP = FiniteDPP('correlation', **{'K_eig_dec': (eig_vals, eig_vecs)})
DPP.plot_kernel()

### b. Exact sampling

In [ ]:
# Sample
for _ in range(10):
    DPP.sample_exact()

DPP.list_of_samples

In [ ]:
DPP.flush_samples()
DPP.list_of_samples

### c. MCMC sampling

`DPPy` has most MCMC samplers from the literature, like the "add-exchange-delete" chain.

In [ ]:
r, N = 4, 10
Phi = randn(r, N)
L = Phi.T.dot(Phi)
DPP = FiniteDPP('likelihood', **{'L': L})

In [ ]:
DPP.flush_samples()
DPP.sample_mcmc('E') # 'E' for "exchange"
DPP.list_of_samples

In [ ]:
eig_vecs, _ = qr(randn(N, r), mode='economic')
eig_vals = rand(r)

DPP = FiniteDPP('correlation', **{'K_eig_dec': (eig_vals, eig_vecs)})
DPP.sample_mcmc('AD')
DPP.list_of_samples

In [ ]:
DPP.flush_samples()
DPP.sample_mcmc('AED')  #E, AD
DPP.list_of_samples

Some MCMC samplers have been designed to allow more global moves, like the "zonotope hit-and-run".

In [ ]:
r, N = 4, 10
A = randn(r, N)

DPP = FiniteDPP('correlation', projection=True, **{'A_zono': A})

DPP.sample_mcmc('zonotope')
print(DPP.list_of_samples)

## 3. $k$-DPPs

In machine learning, $k$-DPPs have also been considered; see [documentation](https://dppy.readthedocs.io/en/latest/finite_dpps/definition.html#k-dpps). One way to define them is condition a DPP defined though its likelihood kernel $\mathbb{L}$ to have a fixed sample size $k$.

$$\mathbb{P}[\mathcal{X} = S] \propto \det \mathbf{L}_S ~~ 1_{|S|=k}.$$

In [ ]:
r, N = 5, 10

# Random feature vectors
Phi = randn(r, N)
DPP = FiniteDPP('likelihood', **{'L': Phi.T.dot(Phi)})

### a. Exact sampling

See [documentation](https://dppy.readthedocs.io/en/latest/finite_dpps/exact_sampling.html#k-dpps).

In [ ]:
k = 3
DPP.flush_samples()
for _ in range(10):
    DPP.sample_exact_k_dpp(size=3)

DPP.list_of_samples

### b. MCMC sampling

To preserve the cardinality of the samples, only exchange moves can be performed among the "add-exchange-delete" options.

In [ ]:
k = 3
DPP.flush_samples()
DPP.sample_mcmc_k_dpp(size=k)
DPP.list_of_samples

## 4. Exotic DPPs

In this section, we gather some exotic DPPs for research purposes. We focus on DPPs that come with apparently adhoc, efficient sampling algorithms. Again, we refer to the [documentation](https://dppy.readthedocs.io/en/latest/exotic_dpps/index.html) for more details and references.

### a. Uniform Spanning Trees

If you draw a uniform spanning tree of a connected graph, and collect its edges in a set, you have a DPP on the set of edges.

In [ ]:
import networkx as nx

from dppy.exotic_dpps import UST

Let us generate a graph.

In [ ]:
g = nx.Graph()
edges = [(0,2), (0,3), (1,2), (1,4), (2,3), (2,4), (3,4)]
g.add_edges_from(edges)

ust = UST(g)

ust.plot_graph()

And display the correlation kernel of the corresponding DPP.

In [ ]:
ust.compute_kernel()
ust.plot_kernel()

Sampling a UST can be done using exact sampling for DPPs, but there are smart Markov chain arguments that yield other *exact* sampling algorithms.

In [ ]:
for md in ('Aldous-Broder', 'Wilson', 'DPP_exact'):
    ust.sample(md)
    ust.plot()

### b. the Poissonized Plancherel measure

This DPP appears in combinatorics and can be sampled using a constructive bijection called the Robinson-Schensted-Knuth correspondence.

In [ ]:
from dppy.exotic_dpps import PoissonizedPlancherel

In [ ]:
theta = 1500  # Poisson parameter
pp_dpp = PoissonizedPlancherel(theta=theta)
pp_dpp.sample()
pp_dpp.plot_diagram(True)

### c. The Carries Process
This DPP is simply obtained by noting the position of carries when you add i.i.d. uniform integer digits in some base $b$.

In [ ]:
from dppy.exotic_dpps import CarriesProcess

In [ ]:
#@title ##### Use a slider!

_base = 10  #@param {type:'slider', min:0, max:10, step:1}
_size = 63  #@param {type:'slider', min:0, max:1000, step:1}

cp = CarriesProcess(_base)

cp.sample(_size)

cp.plot_vs_bernoullis()

# III. Tools behind the scene

---


This collaborative project is hosted on [GitHub](https://github.com/guilgautier/DPPy). **Feel free to contribute!** Start by raising an issue to indicate your suggestion or need.

There is a companion paper [DPPy_paper](https://github.com/guilgautier/DPPy_paper) to this toolbox, please **cite it if you use the toolbox**.

Finally, the documentation is on ReadTheDocs [![Documentation Status](https://readthedocs.org/projects/dppy/badge/?version=latest)](https://dppy.readthedocs.io/en/latest/?badge=latest)
and continuous integration is guaranteed using Travis [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)